In [516]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [517]:
train_csv = 'data/training.csv'
test_csv = 'data/test_X.csv'

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

train_df[:5]

,lettr,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,B,4,8,6,6,5,9,7,4,6,10,5,6,2,8,6,10
1,B,1,0,1,0,0,7,7,6,4,7,6,7,1,8,5,9
2,B,4,8,6,6,8,8,8,4,3,6,7,7,6,11,8,9
3,B,4,7,6,5,5,8,6,5,6,9,6,7,3,8,7,9
4,B,9,14,7,8,5,6,8,5,7,10,6,8,6,6,7,9


In [518]:
X_train = np.array(train_df.iloc[:, 1:])
X_test = np.array(test_df)

In [519]:
y_array = np.array(train_df.iloc[:, 0])
y_chars = np.unique(y_array)
y_dict = dict()
for i, c in enumerate(y_chars):
    y_dict[c] = i
    
y_train = np.array([y_dict[c] for c in y_array])

In [520]:
k = 5
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X_train, y_train)

KNeighborsClassifier()

In [521]:
centroid = dict()
for c in y_chars:
    num = y_dict[c]
    c_list = [X_train[i] for i in range(len(y_train)) if y_train[i] == num]
    c_array = np.array(c_list)
    c_centroid = np.mean(c_array, axis=0)
    centroid[num] = c_centroid

In [522]:
pred = model.predict(X_test)

dist = []
for i in range(len(pred)):
    cen = centroid[pred[i]]
    diff = X_test[i] - cen
    d = np.sqrt(np.sum(np.square(diff)))
    dist.append(d)

d:\python310\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [523]:
indices = [i for i in range(len(dist))]
data = {"id": indices ,"outliers": dist}
out_df = pd.DataFrame(data, columns=["id", "outliers"])
out_df.to_csv('KNN_result.csv', index=False, header=True)

In [524]:
import torch
import torch.nn as nn
import torch.optim as optim

class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [525]:
X_train_char = [X_train[i:i+700] for i in range(0, 4200, 700)]

print(np.array(X_train_char).shape)

(6, 700, 16)


In [526]:
input_dim = X_train.shape[1]
test_data_tensor = torch.Tensor(X_test)
criterion = nn.MSELoss()

num_epochs = 20000
lr = 0.0005

In [527]:
errors = []
for i in range(len(y_chars)):
	
	autoencoder = Autoencoder(input_dim)
	optimizer = optim.Adam(autoencoder.parameters(), lr=lr)
	train_data_tensor = torch.Tensor(X_train_char[i])
	
	for _ in range(num_epochs):
		optimizer.zero_grad()
		outputs = autoencoder(train_data_tensor)
		loss = criterion(outputs, train_data_tensor)
		loss.backward()
		optimizer.step()

	reconstructed_data = autoencoder(test_data_tensor).detach().numpy()
	errors_char = np.mean(np.square(X_test - reconstructed_data), axis=1)
	errors.append(errors_char)
 
min_errors = errors[0]
for cur_error in errors:
	min_errors = np.minimum(min_errors, cur_error)

min_errors = np.round(min_errors, decimals=5)
print(min_errors[:10])

[4.9532  2.35147 1.26303 1.45557 2.54705 6.04212 0.45175 0.13574 1.8253
 3.02296]


In [528]:
indices = [i for i in range(len(min_errors))]
data = {"id": indices ,"outliers": min_errors}
out_df = pd.DataFrame(data, columns=["id", "outliers"])
out_df.to_csv('AE_char_result.csv', index=False, header=True)